# Index创建、查、改、增、删和使用
想要用好pandas，必须了解其核心对象之一的**索引**。
- 索引类似于元组，其本身是不能赋值修改的；
- 其在数据进行整体运算时，辅助自动对齐，这是pandas不同于其他数据处理库的一大特征；
- 多层索引可以帮助改变表的形态，如透视表等。

In [1]:
__auther__ = 'zhenhang.sun@gmail.com'

In [2]:
pwd

'/Users/liuning/project/spark-cap/pandas-cookbook'

In [3]:
import numpy as np
import pandas as pd

# 1. 单层索引

## 1.1 创建
##### `pd.Index(data, dtype=Object, name=None)`
- name：一维列表
- dtype：索引元素的类型，默认为object型
- name：索引的名字，类似于列的名字

In [4]:
data = ['a', 'b', 'c']
index = pd.Index(data, name='name1')
index

Index(['a', 'b', 'c'], dtype='object', name='name1')

**从返回值可以看到，index由三部分组成，可以分别查看。**

In [5]:
index.name

'name1'

In [6]:
index.values

array(['a', 'b', 'c'], dtype=object)

In [7]:
index.dtype

dtype('O')

## 1.2 查
- 查询方式和一维ndarray或Series的.iloc[]完全一样。

In [8]:
index[0] # scalar，返回值

'a'

In [9]:
index[0:2] # 范围，返回index

Index(['a', 'b'], dtype='object', name='name1')

In [10]:
index[[0, 2]] # 列表，返回index

Index(['a', 'c'], dtype='object', name='name1')

In [11]:
mask = [True, False, True]  # mask,返回index
index[mask]

Index(['a', 'c'], dtype='object', name='name1')

## 1.3 改索引名
虽然索引的值是不能修改的，但是名字确是可以修改的。

### 1.3.1 直接改

In [16]:
index.name = 'new_name' 
index

Index(['a', 'b', 'c'], dtype='object', name='new_name')

### 1.3.2 函数改
##### `Index.set_names(names, inplace=False)`
- names：要设置的名字，可以为名字的列表；
- inplace：是否原地修改。

In [17]:
index.set_names('new_name')

Index(['a', 'b', 'c'], dtype='object', name='new_name')

## 1.4 增

### 1.4.1 按位置添加一行
##### `Index.insert(loc, value)`
- loc：位置编号
- value：值

In [18]:
index

Index(['a', 'b', 'c'], dtype='object', name='new_name')

In [19]:
index.insert(1,'d')

Index(['a', 'd', 'b', 'c'], dtype='object', name='new_name')

### 1.4.2 尾部添加多行
##### `Index.append(other)`
- other：其他索引对象

In [20]:
index1 = index.copy()
index1

Index(['a', 'b', 'c'], dtype='object', name='new_name')

In [21]:
index1.append(index)

Index(['a', 'b', 'c', 'a', 'b', 'c'], dtype='object', name='new_name')

### 1.4.2 并集
##### `Index.union(other)`

In [22]:
index2 = pd.Index(['b','c','d'])
index2

Index(['b', 'c', 'd'], dtype='object')

In [23]:
index1.union(index2)

Index(['a', 'b', 'c', 'd'], dtype='object')

## 1.5 删

### 1.5.1 按位置删除一行
##### `Index.delete(loc)`
- loc：位置编号

In [24]:
index1.delete(1)

Index(['a', 'c'], dtype='object', name='new_name')

### 1.5.2 按索引删除多行
##### `Index.drop(labels)`
- labels：索引列表

In [25]:
index1.drop(['a','b'])

Index(['c'], dtype='object', name='new_name')

### 1.5.3 交集
##### `Index.intersection(other)`

In [26]:
index1.intersection(index2) 

Index(['b', 'c'], dtype='object')

# 2. 多层索引

## 2.1 创建
##### `pd.MultiIndex.from_tuples(labels, names=None)`
- labels：元组或列表的列表；
- names：名字的列表。

In [27]:
# data = [['a','one'],['a','two'],['b','one']]
data = [('a','one'),('a','two'),('b','one')]
index = pd.MultiIndex.from_tuples(data, names=['name1','name2'])
index

MultiIndex([('a', 'one'),
            ('a', 'two'),
            ('b', 'one')],
           names=['name1', 'name2'])

In [28]:
s = pd.Series([1,2,3], index = index)
s

name1  name2
a      one      1
       two      2
b      one      3
dtype: int64

## 2.2 查
- 查询方法和单层索引完全一致。

In [29]:
index[0]  # scalar，返回值

('a', 'one')

In [30]:
index[0:2]  # 范围，返回MultiIndex

MultiIndex([('a', 'one'),
            ('a', 'two')],
           names=['name1', 'name2'])

In [31]:
index[[0,2]] # 列表，返回MultiIndex

MultiIndex([('a', 'one'),
            ('b', 'one')],
           names=['name1', 'name2'])

In [32]:
mask = [True, False, True] # mask，返回MultiIndex
index[mask]

MultiIndex([('a', 'one'),
            ('b', 'one')],
           names=['name1', 'name2'])

##### 获取某一层索引 MultiIndex.get_level_values(level)
- level：int，选中的那一层

In [33]:
index.get_level_values(0)

Index(['a', 'a', 'b'], dtype='object', name='name1')

In [34]:
index.get_level_values(1)

Index(['one', 'two', 'one'], dtype='object', name='name2')

## 2.3 改

### 2.3.1 改索引名
##### `MultiIndex.set_names(names, level=None, inplace=False)`
- names：要设置的名字，可以为名字的列表；
- level：多层索引需要设置修改的索引层次，可以为列表，要与names匹配；
- inplace：是否原地修改。

In [35]:
index.set_names('new_name_1',level=0)

MultiIndex([('a', 'one'),
            ('a', 'two'),
            ('b', 'one')],
           names=['new_name_1', 'name2'])

### 2.3.2 改索引层次顺序
##### `MultiIndex.swaplevel(i=-2, j=-1)`
- 改变level i 和level j的次序

In [36]:
index.swaplevel()

MultiIndex([('one', 'a'),
            ('two', 'a'),
            ('one', 'b')],
           names=['name2', 'name1'])

##### `Series.swaplevel(i=-2, j=-1)`
#####  `DataFrame.swaplevel(i=-2, j=-1, axis=1)`
- axis：0-行索引，1-列索引。
这两个函数更实用一些。

In [37]:
s.swaplevel()

name2  name1
one    a        1
two    a        2
one    b        3
dtype: int64

In [38]:
columns = index.copy()
columns.set_names( names = ['name3','name4'], level = [0,1], inplace = True) #列索引取和行索引相同，只是改了名字
df = pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]], index= index, columns = columns)
df

name3         a       b
name4       one two one
name1 name2            
a     one     1   2   3
      two     4   5   6
b     one     7   8   9

In [39]:
df.swaplevel(axis=1)  # 交换列索引顺序

name4       one two one
name3         a   a   b
name1 name2            
a     one     1   2   3
      two     4   5   6
b     one     7   8   9

# 3. 多层索引使用方法
当对values进行查看时，多层索引可以分开使用。

In [40]:
df1 = df.copy()
df1

name3         a       b
name4       one two one
name1 name2            
a     one     1   2   3
      two     4   5   6
b     one     7   8   9

**索引为空不代表缺失，缺省写法，意思是之前的索引一致。**

## 3.1 对于外层索引
**记住：**
- 无论是Series还是DataFrame，外层索引都是可以直接使用，也就是说可以认为只有这一层索引；
- **用法和第二篇查、改、增、删提到的方法完全相同**。

### 3.1.1 []
快捷操作，还是四种用法。

In [41]:
df1

name3         a       b
name4       one two one
name1 name2            
a     one     1   2   3
      two     4   5   6
b     one     7   8   9

In [42]:
df1['b']  # 列外层

name4        one
name1 name2     
a     one      3
      two      6
b     one      9

In [43]:
df1[['a','b']] # 列外层

name3         a       b
name4       one two one
name1 name2            
a     one     1   2   3
      two     4   5   6
b     one     7   8   9

In [44]:
df1[0:2]  # 行外层

name3         a       b
name4       one two one
name1 name2            
a     one     1   2   3
      two     4   5   6

In [45]:
mask =[True, False, True]  # 行外层
df1[mask]

name3         a       b
name4       one two one
name1 name2            
a     one     1   2   3
b     one     7   8   9

### 3.1.2  .loc[]
行和列索引都使用索引形式。

**下面都以第一维度为例，第二维可以类比。**

In [46]:
df1.loc['a','b']  # 单行索引'a' 

name4,one
name2,
one,3
two,6


In [47]:
df1.loc['a':'b', 'b']   #范围'a': 'b'

name4        one
name1 name2     
a     one      3
      two      6
b     one      9

In [48]:
df1.loc[['a','b'], 'b'] #列表

name4        one
name1 name2     
a     one      3
      two      6
b     one      9

In [49]:
mask = [True, False, True] # mask
df1.loc[mask,'b']

,name4,one
name1,name2,
a,one,3
b,one,9


### 3.1.3 .iloc[]
这个简单，可以把索引都忽略掉，行和列都使用位置形式。

In [50]:
df1.iloc[0,0:2] 

name3  name4
a      one      1
       two      2
Name: (a, one), dtype: int64

In [51]:
df1.iloc[0:2, 0:2] 

name3         a    
name4       one two
name1 name2        
a     one     1   2
      two     4   5

In [52]:
df1.iloc[[0,1],0:2] 

name3         a    
name4       one two
name1 name2        
a     one     1   2
      two     4   5

In [53]:
mask = [True, False, True]
df1.iloc[mask,0:2]

name3         a    
name4       one two
name1 name2        
a     one     1   2
b     one     7   8

## 3.2 对于内层索引
- **内层索引不可直接使用，必须先外层、再内层，直接使用会报错；**
- 内层只能使用单索引形式，其他形式报错。

### 3.2.1 [ , ]
快捷操作，只有一种用法，取出一列。

In [54]:
df1

name3         a       b
name4       one two one
name1 name2            
a     one     1   2   3
      two     4   5   6
b     one     7   8   9

In [55]:
df1['a','one']  #取一列，先外层单列索引，再内层单列索引，其他形式都报错

name1  name2
a      one      1
       two      4
b      one      7
Name: (a, one), dtype: int64

### 3.2.2 .loc[ , ]

In [56]:
df1.loc['a','one'] # 取一行，先外层单行索引，再内层单列索引，其他形式都报错

name3  name4
a      one      1
       two      2
b      one      3
Name: (a, one), dtype: int64

### 3.2.3 .iloc[ , ]
这种方法不受影响，因为 .iloc[] 无视索引，只按照位置定位，所以和3.1.3节外层索引部分完全相同。

## 3.3 xs直接选取法
适合在单层level选取，不能行列同时操作。
##### `Series.xs(key, level=None, drop_level=True)`
##### `DataFrame.xs(key, axis=0, level=None, drop_level=True)`
- key: 要选取的索引值或其列表；
- axis：0-行索引，1-列索引；
- level：索引层次；
- drop_level：True or False，是否显示用于选取的level索引，默认不显示。

In [53]:
df1 = df.copy()
df1

name3         a       b
name4       one two one
name1 name2            
a     one     1   2   3
      two     4   5   6
b     one     7   8   9

In [54]:
df1.xs('one', axis=0, level=1)  # 行索引的level 1, 有两行

name3   a       b
name4 one two one
name1            
a       1   2   3
b       7   8   9

In [55]:
df1.xs('two', axis=1, level=1) # 列索引的level 1，有一列

name3        a
name1 name2   
a     one    2
      two    5
b     one    8